In [1]:
import numpy as np
import pandas as pd
import random
import os

# 1. Generate parameters

The paper found that based on the curve-fitting parameters `Gb`, `tau`, `alpha`, `omega`, `C0`, calculate `w0` = $\sqrt{\omega^2 + \alpha^2}$ and `TO` = $\frac{2 \pi}{\omega_0}$:
- If `T0` > 4hr: a person is prediabetic or diabetic
- A person is normal otherwise

Moreover, based on the curve-fitting parameters from the paper for 2 patients A (normal) and B (diabetic), we will take reference from these values to set the range for each fit parameters. 

Here are the values for fit parameters of each person and their diabetic status from the paper and range of each parameter chosen:

|Person|Diabetic status| `Gb`|`alpha`|`tau`|`omega`|`C0`|`T0`|`Gb` range|`alpha` range|`tau` range|`C0` range|`T0` range|
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|A|normal|73.8161|1.1733|0.6447|2.4128|116.4327|2.3418|[70, 85]|[1, 1.3]|[0.6, 0.7]|[100, 130]|[2, 4]|
|B|diabetic|94.4838|0.8685|0.6447|1.2823|208.3634|4.0569|[85, 120]|[0.1, 1]|[0.6, 0.7]|[130, 210]|[4, 10]|

In our function `generate_parameters`, it will take in `patient_type` as input. 
- Based on the patient type, it will generate random parameters `Gb`, `tau`, `alpha`, `C0`,`T0` in a given range.
- Then, it calculates parameters `w0` and `w0`.
- The function returns fit parameters `Gb`, `tau`, `alpha`, `omega`, `C0` used for the mathematical function $G(t)$.
- Note: for ease of calculation, we round all parameters to 2 digits.

In [2]:
def generate_parameters(patient_type):
    #set range of parameters based on patient type
    if patient_type == 'diabetic':
        min_Gb, max_Gb = 85, 120
        min_T0, max_T0 = 4, 10
        min_alpha, max_alpha = 0.1,1
        min_C0, max_C0 = 130, 210
    elif patient_type == 'normal':
        min_Gb, max_Gb = 70, 85
        min_T0, max_T0 = 2,4
        min_alpha, max_alpha = 1,1.3
        min_C0, max_C0 = 100,130
    
    #generate random parameters 
    Gb = round(random.uniform(min_Gb, max_Gb), 2)
    T0 = round(random.uniform(min_T0, max_T0), 2)
    w0 = round(2 * np.pi/T0,2)
    alpha = round(random.uniform(min_alpha, max_alpha), 2)
    omega = round((w0**2 - alpha**2),2)
    C0 = round(random.uniform(min_C0, max_C0),2)
    tau = round(random.uniform(0.6,0.7), 2)

    return Gb, alpha, omega, C0, tau

# 2. Function to generate simulation

Note that the blood-glucose level $G$ varies with time $t$ according to the mathematical function:
$$
G(t) = G_b + D_2 = G_b + e^{-\alpha t} \left[ -C_0 \cos(\omega t) + \frac{A_G D + C_0 - \alpha C_0 \tau}{\omega \tau} \sin(\omega t) \right] + C_0 e^{\frac{-t}{\tau}}.
$$

with known parameters $G_b, \alpha, \omega, C_0, \tau$.

- We will use the randomly generated and calculated parameters `Gb`, `tau`, `alpha`, `omega`, `C0` from the function `generate_parameters` to apply to the mathematical function. 
- We use the same range of sampled time `t` like that of the sample dataset in the paper.
- Apply the mathematical function $G(t)$ with the parameters $G_b, \alpha, \omega, C_0, \tau$ at each point of time $t$.
- We add a small noise in the range [-1, 1] to the function $G(t)$.
- Save the simulated glucose level $G$ and time $t$ as a Dataframe
- Save the Dataframe as a csv file in `simulation` folder
- Follow the naming convention of the file

In [3]:
def generate_simulation(patient_type, number_of_datasets):
    food = 100
    for index in range(number_of_datasets):
        #generate parameters
        Gb, alpha, omega, C0, tau = generate_parameters(patient_type)

        t = [0.00,0.50, 0.75, 1.00, 1.50, 2.00, 2.50, 3.00, 4.00, 6.00]
        
        G = []
        j = 0
        
        #for each t, generate G(t) using mathematical function with a little noise 
        for t_i in t:
            try:
                noise = random.uniform(-1,1)
                glucose = Gb + np.exp(-alpha*t_i)*(-C0*np.cos(omega*t_i)+ (food+C0-alpha*C0*tau)/(omega*tau)*np.sin(omega*t_i)) +C0*np.exp(-t_i/tau) + noise
                math = Gb + np.exp(-alpha*t_i)*(-C0*np.cos(omega*t_i)+ (food+C0-alpha*C0*tau)/(omega*tau)*np.sin(omega*t_i)) +C0*np.exp(-t_i/tau) 
                G.append(glucose)

            except ZeroDivisionError:
                print(t_i)
        t_array = np.arange(min(t), max(t), 0.01)
        M = Gb + np.exp(-alpha*t_array)*(-C0*np.cos(omega*t_array)+ (food+C0-alpha*C0*tau)/(omega*tau)*np.sin(omega*t_array)) +C0*np.exp(-t_array/tau) 
        df = pd.DataFrame(zip(t,G), columns = ['t', 'G(t)'])
        
        # Create the folder for simulation data if it doesn't exist
        folder_path = f'./simulation/{patient_type}'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # naming convention of each csv file: '{patient_type}_{index}.csv' (e.g normal_1.csv, diabetic_1.csv)
        # Save DataFrame to CSV in the simulation folder
        file_name = f'{patient_type}_{index+1}.csv'
        file_path = os.path.join(folder_path, file_name)

        df.to_csv(file_path, index=False) #uncomment this
        #return t

## 2.1 Generate simulation for diabetic patients

In [4]:
generate_simulation('diabetic', 50)

## 2.2 Generate simulation for normal patients

In [5]:
generate_simulation('normal', 50)